In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline
matplotlib.rcParams["figure.figsize"] = (20,10)

KeyboardInterrupt: 

In [ ]:
df1 = pd.read_csv("C:\Users\abc\Desktop\Projects\House_Price_Prediction\HPP(Dataset).csv")
df1.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/Bengaluru_House_Data.csv'

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
df1.shape

In [ ]:
df1.columns

In [ ]:
df2 = df1.drop(['area_type','availability','society','balcony'],axis='columns')
df2.shape

In [ ]:
df2.head()

In [ ]:
df2.isnull().sum()

In [ ]:
df3 = df2.dropna()
df3.isnull().sum()

In [ ]:
df3['size'].unique()

In [ ]:
df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))

In [ ]:
df3.head()

In [ ]:
df3.bhk.unique()

In [ ]:
df3[df3.bhk>10]

In [ ]:
df3.total_sqft.unique()

NameError: name 'df3' is not defined

In [ ]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
df3[~df3['total_sqft'].apply(is_float)]

In [ ]:
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
      return float(x)
    except:
      return None

In [ ]:
convert_sqft_to_num('2345')

In [ ]:
convert_sqft_to_num('2100 - 2500')

In [ ]:
df4=df3.copy()
df4['total_sqft'] = df4['total_sqft'].apply(convert_sqft_to_num)
df4.head()

In [ ]:
df4.loc[30]

In [ ]:
df5=df4.copy()
df5['price_per_sqft'] = df5['price']*100000/df5['total_sqft']
df5.head()

In [ ]:
df5.location.unique()

In [ ]:
len(df5.location.unique())

In [ ]:
df5.location=df5.location.apply(lambda x: x.strip())
location_stats = df5['location'].value_counts(ascending=False)
location_stats

In [ ]:
len(location_stats[location_stats>10])

In [ ]:
len(location_stats[location_stats<=10])

In [ ]:
len(df5.location.unique())

In [ ]:
location_stats_less_than_10 = location_stats[location_stats<=10]
location_stats_less_than_10

In [ ]:
df5.location=df5.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x)
len(df5.location.unique())

In [ ]:
df5.head(20)

In [ ]:
df5[df5.total_sqft/df5.bhk<300].head()

In [ ]:
df6=df5[~(df5.total_sqft/df5.bhk<300)]
df6.shape

In [ ]:
df6.price_per_sqft.describe()

In [ ]:
def remove_ppf_outliers(df):
  df_out=pd.DataFrame()
  for key,subdf in df.groupby('location'):
    m=np.mean(subdf.price_per_sqft)
    st=np.std(subdf.price_per_sqft)
    reduce_df=subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
    df_out=pd.concat([df_out,reduce_df],ignore_index=True)
  return df_out

df7=remove_ppf_outliers(df6)
df7.shape

In [ ]:
def plot_scatter_chart(df,location):
    bhk2 = df[(df.location==location) & (df.bhk==2)]
    bhk3 = df[(df.location==location) & (df.bhk==3)]
    matplotlib.rcParams['figure.figsize'] = (15,10)
    plt.scatter(bhk2.total_sqft,bhk2.price,color='blue',label='2 BHK', s=50)
    plt.scatter(bhk3.total_sqft,bhk3.price,marker='+', color='red',label='3 BHK', s=50)
    plt.xlabel("Total Square Feet Area")
    plt.ylabel("Price (Lakh Indian Rupees)")
    plt.title(location)
    plt.legend()

In [ ]:
plot_scatter_chart(df7,"Hebbal")

In [ ]:
plot_scatter_chart(df7,"Electronic City")

In [ ]:
def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')

In [ ]:
df8 = remove_bhk_outliers(df7)
df8.shape

In [ ]:
plot_scatter_chart(df8,"Hebbal")

In [ ]:
plot_scatter_chart(df8,"Electronic City")

In [ ]:
matplotlib.rcParams["figure.figsize"] = (20,10)
plt.hist(df8.price_per_sqft,rwidth=0.8)
plt.xlabel("Price Per Square Feet")
plt.ylabel("Count")

In [ ]:
df8.bath.unique()

In [ ]:
df8[df8.bath>8]

In [ ]:
plt.hist(df8.bath,rwidth=0.8)
plt.xlabel("Number of bathrooms")
plt.ylabel("Count")

In [ ]:
df8[df8.bath>df8.bhk+2]

In [ ]:
df9=df8[df8.bath<df8.bhk+2]
df9.shape

In [ ]:
df10=df9.drop(['size','price_per_sqft'],axis='columns')
df10.head()

NameError: name 'df9' is not defined

In [ ]:
dummies = pd.get_dummies(df10.location)
dummies.head()

In [ ]:
df11=pd.concat([df10,dummies.drop('other',axis='columns')],axis='columns')
df11.head()

In [ ]:
df12=df11.drop('location',axis='columns')
df12.head()

In [ ]:
df12.shape

NameError: name 'df12' is not defined

In [ ]:
X = df12.drop('price',axis='columns')
X.head()

In [ ]:
X.shape

In [ ]:
y = df12.price
y.head()

NameError: name 'df12' is not defined

In [ ]:
len(y)

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=10)

In [ ]:

# import numpy as np
# import warnings
# from sklearn.datasets import fetch_california_housing
# from sklearn.model_selection import cross_val_score, ShuffleSplit
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import StandardScaler


# from sklearn.linear_model import LinearRegression
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.neighbors import KNeighborsRegressor
# from xgboost import XGBRegressor


# warnings.filterwarnings('ignore')




# X, y = fetch_california_housing(return_X_y=True)

# cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)



# print("--- Evaluating Linear Regression ---")
# lr_scores = cross_val_score(LinearRegression(), X, y, cv=cv)
# print(f"Scores for each fold: {lr_scores}")
# print(f"Average R-squared score: {np.mean(lr_scores):.4f} (+/- {np.std(lr_scores):.4f})\n")


# print("--- Evaluating Random Forest ---")
# rf_scores = cross_val_score(RandomForestRegressor(random_state=0), X, y, cv=cv)
# print(f"Scores for each fold: {rf_scores}")
# print(f"Average R-squared score: {np.mean(rf_scores):.4f} (+/- {np.std(rf_scores):.4f})\n")



# print("--- Evaluating K-Nearest Neighbors (with Scaling) ---")
# knn_pipeline = Pipeline([
#     ('scaler', StandardScaler()),
#     ('knn', KNeighborsRegressor(n_neighbors=7))
# ])
# knn_scores = cross_val_score(knn_pipeline, X, y, cv=cv)
# print(f"Scores for each fold: {knn_scores}")
# print(f"Average R-squared score: {np.mean(knn_scores):.4f} (+/- {np.std(knn_scores):.4f})\n")



# print("--- Evaluating XGBoost ---")
# xgb_scores = cross_val_score(XGBRegressor(random_state=0), X, y, cv=cv)
# print(f"Scores for each fold: {xgb_scores}")
# print(f"Average R-squared score: {np.mean(xgb_scores):.4f} (+/- {np.std(xgb_scores):.4f})\n")

In [ ]:
# from sklearn.linear_model import LinearRegression
# lr_clf = LinearRegression()
# lr_clf.fit(X_train,y_train)
# lr_clf.score(X_test,y_test)

In [ ]:
# from sklearn.model_selection import ShuffleSplit
# from sklearn.model_selection import cross_val_score

# cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)

# cross_val_score(LinearRegression(), X, y, cv=cv)

In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, ShuffleSplit

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor


def find_best_model_using_gridsearchcv(x, y):


    algos = {
        'linear_regression': {
            'model': LinearRegression(),
            'params': {

            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1, 2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion': ['squared_error', 'friedman_mse'],
                'splitter': ['best', 'random']
            }
        },
        'random_forest': {
            'model': RandomForestRegressor(),
            'params': {
                'n_estimators': [50, 100, 150],
                'max_depth': [None, 10, 20]
            }
        },
        'knn': {
            'model': KNeighborsRegressor(),
            'params': {
                'n_neighbors': [3, 5, 7, 9],
                'weights': ['uniform', 'distance']
            }
        },
        'xgboost': {
            'model': XGBRegressor(objective='reg:squarederror'),
            'params': {
                'n_estimators': [50, 100, 150],
                'learning_rate': [0.05, 0.1, 0.2]
            }
        }
    }

    scores = []

    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)


    for algo_name, config in algos.items():
        print(f"Running GridSearchCV for {algo_name}...")
        gs = GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False, n_jobs=-1)
        gs.fit(x, y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])

In [ ]:
find_best_model_using_gridsearchcv(X, y)

In [ ]:
print("\nTraining the final chosen model (XGBoost)...")
final_model = XGBRegressor(n_estimators=150, learning_rate=0.1, random_state=0)
final_model.fit(X, y)
print("Final model trained successfully.")




def predict_price(location, sqft, bath, bhk):
    """Predicts price for Bengaluru properties using the final trained model."""
    try:
        loc_index = np.where(X.columns == location)[0][0]
    except IndexError:
        loc_index = -1

    x_input = np.zeros(len(X.columns))
    x_input[0] = sqft
    x_input[1] = bath
    x_input[2] = bhk
    if loc_index >= 0:
        x_input[loc_index] = 1

    predicted_price = final_model.predict([x_input])[0]
    return round(float(predicted_price), 2)

In [ ]:
# def predict_price(location,sqft,bath,bhk):
#     loc_index = np.where(X.columns==location)[0][0]

#     x = np.zeros(len(X.columns))
#     x[0] = sqft
#     x[1] = bath
#     x[2] = bhk
#     if loc_index >= 0:
#         x[loc_index] = 1

#     return lr_clf.predict([x])[0]

In [ ]:
price1 = predict_price('1st Phase JP Nagar', 1000, 2, 2)
print(f"Predicted price for 1000 sqft, 2 bed, 2 bath in 1st Phase JP Nagar: {price1} Lakhs")

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor


dummies = pd.get_dummies(df10.location)
df_final = pd.concat([df10.drop('location', axis=1), dummies.drop('other', axis=1)], axis=1)

X = df_final.drop('price', axis=1)
y = df_final['price']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=0)
xgb_model.fit(X_train, y_train)


y_pred = xgb_model.predict(X_test)

plt.figure(figsize=(10, 8))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=2)
plt.title('Actual vs. Predicted Prices (XGBoost)', fontsize=16)
plt.xlabel('Actual Price (in Lakhs)', fontsize=12)
plt.ylabel('Predicted Price (in Lakhs)', fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.savefig('xgb_actual_vs_predicted.png')


In [ ]:

from sklearn.metrics import mean_absolute_percentage_error


y_pred = xgb_model.predict(X_test)

#  Mean Absolute Percentage Error
mape = mean_absolute_percentage_error(y_test, y_pred)


print(f"\nMean Absolute Percentage Error (MAPE): {mape:.2%}")
print(f"This means, on average, the model's prediction is off by {mape:.2%} from the actual price.")

In [ ]:

# R-squared score (which is a decimal)
r2_score = xgb_model.score(X_test, y_test)

print(f"R-squared Score (as a decimal): {r2_score:.4f}")
print(f"This means the model explains {r2_score:.2%} of the variance in the house prices.")

NameError: name 'xgb_model' is not defined

In [ ]:
# predict_price('1st Phase JP Nagar',1000, 2, 2)